## はじめに
本ハンズオンではSageMakerの組み込みアルゴリズムを活用した機械学習を体験頂きます。学習に使うデータをAmazon S3のバケットへアップロードし、組み込みアルゴリズムとSageMakerの学習用インスタンスを使ってモデルを学習、学習されたモデルを推論インスタンスへデプロイする流れをご確認下さい。

今回はテキストデータについて類似の単語群を探すというタスクを下記の手順で行います。
- wikipedia由来のデータ用いてWord2Vecモデルを学習
- Word2Vecモデルが適切に学習されているかを確認
    - テキストデータをWrod2Vecで100次元のベクトル表現へ変換
    - 100次元のベクトルをt-SNEを用いて2次元のベクトルへ次元圧縮
    - 2次元データを散布図で可視化し定性的に評価
- 100次元データをK-Meansアルゴリズムを用いてクラスタリングし、似た表現のグループを作成
- 未知のデータをWord2Vecで100次元ベクトル化した上で、上述のどのグループに所属するかを計算

今後のプロトタイピングを実施するに辺り、本ハンズオンをとおしてご確認を頂きたい概念は下記です。
- SageMakerの組み込みアルゴリズムを用いた機械学習の実施
- テキストデータをベクトルへ変換するとはどういうことか
- 次元圧縮するとはどういうことか
- クラスタリングとは何か

逆に、現時点で時間をかけて頂かなくて良い点は下記になります。
- 機械学習アルゴリズムそのもの(今回で言えばWord2Vec、t-SNE、K-Meansなど自体）
- Pythonライブラリのそれぞれの文法や詳細(numpy, pandas, scikit-learn、SageMakerなど)
- 今回はほとんど行っていない自然言語処理におけるデータの前処理(ノイズの除去や単語の正規化、ストップワードの除去など)

### データの準備
今回はWikipediaから抽出した文章に対してクリーニングなどの処理をした後の[ja.text8](https://github.com/Hironsan/ja.text8)という100MB程度のデータ(コーパス)を活用します。

In [ ]:
# データのダウンロードと解凍
!wget https://s3-ap-northeast-1.amazonaws.com/dev.tech-sketch.jp/chakki/public/ja.text8.zip
!unzip -o ja.text8.zip

データを読み込み、中身を確認します。総単語数、ユニークな単語数、文書を見ます。

In [ ]:
import sagemaker
from sagemaker import get_execution_role
import boto3
import json

sess = sagemaker.Session()
role = get_execution_role()


# ファイルの読み込み
with open("ja.text8") as f:
    words = f.read().split()
    
print("総単語数:{}".format(len(words)))
print("ユニークな単語数：{}".format(len(set(words))))

In [ ]:
# どのような文章か先頭300単語を表示
"".join(words[:300])

# データのアップロード
SageMakerの学習インスタンスが利用できるよう、データをS3のバケットへアップロードします。

In [ ]:
# アップロード先を指定
prefix = 'sagemaker/DEMO-blazingtext-text8' 
train_channel = prefix + '/train'

# S3へアップロード
bucket = sess.default_bucket() # Replace with your own bucket name if needed
sess.upload_data(path='ja.text8', bucket=bucket, key_prefix=train_channel)

# アップロード先のパス
s3_train_data = 's3://{}/{}'.format(bucket, train_channel)
s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)

# Word2Vecアルゴリズムの学習
今回はSageMakerのWord2Vecの組み込みアルゴリズムであるBlazingTextを活用します。

In [ ]:
region_name = boto3.Session().region_name
container = sagemaker.amazon.amazon_estimator.get_image_uri(region_name, "blazingtext", "latest")


# どのような学習を行うかの設定
bt_model = sagemaker.estimator.Estimator(container,
                                         role, 
                                         train_instance_count=2, 
                                         train_instance_type='ml.c4.2xlarge',
                                         train_volume_size = 5,
                                         train_max_run = 360000,
                                         input_mode= 'File',
                                         output_path=s3_output_location,
                                         sagemaker_session=sess)


# Word2Vecアルゴリズムのハイパラ設定
bt_model.set_hyperparameters(mode="batch_skipgram",
                             epochs=5,
                             min_count=5,
                             sampling_threshold=0.0001,
                             learning_rate=0.05,
                             window_size=5,
                             vector_dim=100,
                             negative_samples=5,
                             batch_size=11, #  = (2*window_size + 1) (Preferred. Used only if mode is batch_skipgram)
                             evaluation=True,# Perform similarity evaluation on WS-353 dataset at the end of training
                             subwords=False) # Subword embedding learning is not supported by batch_skipgram

# 学習データの指定
train_data = sagemaker.session.s3_input(s3_train_data,
                                        distribution='FullyReplicated',
                                        content_type='text/plain',
                                        s3_data_type='S3Prefix')
data_channels = {'train': train_data}


# 学習の開始
bt_model.fit(inputs=data_channels, logs=True)

# Word2Vecを使って単語のベクトル表現を得る
まずは学習されたWord2Vecを推論エンドポイントへデプロイして、推論(単語のベクトル表現化)ができる状態にします。

In [ ]:
bt_endpoint = bt_model.deploy(initial_instance_count = 1,instance_type = 'ml.m4.xlarge')

準備ができたので単語をベクトルに変換してみます。

In [ ]:
# 自転車や女性という単語を推論するようリクエストを作成
words = ["自動車", "女性"]
request = json.dumps({"instances" : words})

# リクエストを推論エンドポイントへ投げ、レスポンスを得る
response = bt_endpoint.predict(request)

# レスポンスの中身を確認
vecs = json.loads(response)
print(vecs[0])

In [ ]:
print(vecs[1])

# Word2Vecの評価
ja.text8で学習したモデルが適切に単語をベクトル化できているか確認するために、ja.text8の単語群のベクトル表現(100次元ベクトル)を入手し、t-SNEアルゴリズムで2次元のベクトル表現まで次元圧縮を行った上で、散布図を描いて可視化します。

In [ ]:
s3 = boto3.resource('s3')
key = bt_model.model_data[bt_model.model_data.find("/", 5)+1:]
s3.Bucket(bucket).download_file(key, 'model.tar.gz')

ダウンロードしてきた学習済モデルのベクトル表現を解凍します。`vector.txt`がベクトル表現です。

In [ ]:
!tar -xvzf model.tar.gz

ベクトル表現はデータは頻度での降順で保存されているため上位400データを可視化対象とします。

In [ ]:
import numpy as np
from sklearn.manifold import TSNE
from sklearn.preprocessing import normalize

# 使用するデータ数を400と指定
num_points = 400

# 可視化しやすいよう前処理を行います。
first_line = True
index_to_word = []
with open("vectors.txt","r") as f:
    for line_num, line in enumerate(f):
        if first_line:
            dim = int(line.strip().split()[1])
            word_vecs = np.zeros((num_points, dim), dtype=float)
            first_line = False
            continue
        line = line.strip()
        word = line.split()[0]
        vec = word_vecs[line_num-1]
        for index, vec_val in enumerate(line.split()[1:]):
            vec[index] = float(vec_val)
        index_to_word.append(word)
        if line_num >= num_points:
            break
word_vecs = normalize(word_vecs, copy=False, return_norm=False)

# t-SNEアルゴリズムのハイパラ設定
tsne = TSNE(perplexity=40, n_components=2, init='pca', n_iter=10000)
# 100次元ベクトルから2次元ベクトルへ変換
two_d_embeddings = tsne.fit_transform(word_vecs[:num_points])
labels = index_to_word[:num_points]

可視化のための準備として日本語フォントをダウンロードします。

In [ ]:
!sudo yum install -y ipa-gothic-fonts

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib as mpl
 
mpl.font_manager._rebuild() #キャッシュの削除
plt.rcParams['font.family'] = 'IPAGothic' # インストールしたフォントを指定


def plot(embeddings, labels):
    plt.figure(figsize=(20,20))
    for i, label in enumerate(labels):
        x, y = embeddings[i,:]
        plt.scatter(x, y)
        plt.annotate(label, xy=(x, y), xytext=(5, 2), textcoords='offset points',
                       ha='right', va='bottom')
    plt.savefig('image.png')
    plt.show()

plot(two_d_embeddings, labels)

# 単語のクラスタリング
ベクトル化された単語群をK-Meansアルゴリズムを使ってクラスタリングしグループを作ります。K-Meansに関してもSageMakerの組み込みアルゴリズムを使います。
Word2Vecの時と同様にS3バケットへデータ(100次元ベクトル表現)をアップロードし、モデルを学習、推論エンドポイントへモデルをデプロイした上でモデル評価を行います。

In [ ]:
from sagemaker import KMeans

# K-Meansアルゴリズムに学習できる形式へデータタイプを変換
word_vecs32 = word_vecs.astype('float32')

# アップロード先のパス
data_location = 's3://{}/kmeans_highlevel_example/data'.format(bucket)
output_location = 's3://{}/kmeans_example/output'.format(bucket)

# K-Meansアルゴリズムの設定
kmeans = KMeans(role=role,
                train_instance_count=2,
                train_instance_type='ml.c4.xlarge',
                output_path=output_location,
                k=10,
                data_location=data_location)

# 組み込みアルゴリズム用のデータ形式へ変換してS3へアップロード
inputs = kmeans.record_set(word_vecs32)

# 学習の開始
kmeans.fit(inputs)

# クラスタリングモデルの評価
まずは学習されたK-Meansモデルを推論エンドポイントへデプロイして、推論(ベクトル表現のグループ化)ができる状態にします。

In [ ]:
kmeans_endpoint = kmeans.deploy(initial_instance_count=1,
                                instance_type='ml.m4.xlarge')

今回は400個の単語がを10個のグループにクラスタリングしました。どのような単語がどのグループに所属するか確認しましょう。

In [ ]:
# 学習用に使ったデータ(10次元ベクトル表現データ)を推論に使い、クラスを得る。
result = kmeans_endpoint.predict(word_vecs32)

clusters = [r.label['closest_cluster'].float32_tensor.values[0] for r in result]
for cluster in range(10):
    words = [word for l, word in zip(clusters, labels) if int(l) == cluster]
    print('クラスタ{}に属する単語は下記となります'.format(cluster))
    print(words)
    print('============================================')

## 入力する任意の文字列に対するクラスタリング
新しい単語に対してはWord2Vecでベクトル化した上で、K-Meansにてどのグループに所属するかを計算します。

In [ ]:
# 新しい単語を入力します。
input_word = '自動車'
new_word = [input_word]

# Word2Vecの推論エンドポイントへリクエストとして入力しベクトル表現を得る
payload = {"instances" : new_word}
response = bt_endpoint.predict(json.dumps(payload))
new_vec = normalize(np.array(json.loads(response)[0]['vector']).reshape(1, -1), copy=False, return_norm=False).astype('float32')

# K-Meansの推論エンドポイントへベクトル表現を入力し所属するグループを得る
result = kmeans_endpoint.predict(new_vec)

# 同じクラスに所属する単語を表示する
similar_words = [word for l, word in zip(clusters, labels) if int(l) == int(result[0].label['closest_cluster'].float32_tensor.values[0])]
print('入力された文字列は「{}」です'.format(input_word))
print('====類似性の近い単語群は下記です。＝＝＝＝＝＝')
print(similar_words)

## 推論エンドポイントの削除
エンドポイントは起動したままだとコストがかかります。不要な場合は削除します。

In [ ]:
sagemaker.Session().delete_endpoint(bt_endpoint.endpoint)
sagemaker.Session().delete_endpoint(kmeans_endpoint.endpoint)

## 参考
- [機械学習のための特徴量エンジニアリング](https://www.oreilly.co.jp/books/9784873118680/)
    - 3章テキストデータの取り扱い
    - 4章特徴量スケーリングによる効果
- [自然言語処理における前処理の種類とその威力](https://qiita.com/Hironsan/items/2466fe0f344115aff177)